In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, silhouette_score
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/monicabellare/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
wordsToCut = set(stopwords.words('english'))
wordsToCut

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [6]:
print ('A' in wordsToCut)

False


In [8]:
#data = pd.read_csv('/Users/daviddeng8/affinity/data/outlet_data.csv')
#data['content'][:100]
data = pd.read_csv('/Users/monicabellare/CL/affinity/data/final_data.csv')
#data.loc[261, 'Url']
data

,Url,Author,Date,Header,Body,n_links,Source,Bias,Quality
0,https://abcnews.go.com/Politics/abortion-right...,Devin Dwyer,"Wed, 17 Apr 2019 10:14:00 GMT",Abortion rights group asks Supreme Court to st...,Abortion rights advocates have asked the U.S. ...,3.0,ABC,1.67,49.00
1,https://abcnews.go.com/Politics/appeals-court-...,Ali Dukakis,"Tue, 26 Feb 2019 09:05:00 GMT",Appeals court says special counsel Robert Muel...,A federal appeals court rejected the most dire...,2.0,ABC,0.67,51.67
2,https://abcnews.go.com/Politics/attorney-gener...,Luke Barr,"Wed, 17 Apr 2019 14:02:00 GMT",Attorney general orders some asylum seekers to...,As part of the Trump administration's effort t...,6.0,ABC,-2.75,43.50
3,https://abcnews.go.com/Politics/donald-trump-t...,Meridith McGraw,"Tue, 19 Mar 2019 12:44:00 GMT","Donald Trump and 'the Trump of the Tropics,' B...","President Donald Trump and ""the Trump of the T...",10.0,ABC,-4.33,52.67
4,https://abcnews.go.com/Politics/electoral-coll...,Matthew Dowd,"Tue, 19 Mar 2019 21:39:00 GMT",The Electoral College limits the campaign play...,"U.S Senator Elizabeth Warren, who is competing...",5.0,ABC,-10.00,32.00
5,https://abcnews.go.com/Politics/facebook-agree...,Anne Flaherty,"Tue, 19 Mar 2019 13:38:00 GMT",Facebook agrees to hide demographics from land...,Facebook announced Tuesday that it will block ...,6.0,ABC,-2.33,52.33
6,https://abcnews.go.com/Politics/learned-2020-c...,Benjamin Siegel \nand,"Wed, 17 Apr 2019 01:41:00 GMT",What we've learned from 2020 candidates' tax r...,A handful of Democratic candidates for preside...,14.0,ABC,-0.67,47.67
7,https://abcnews.go.com/Politics/michael-cohen-...,Aaron Katersky,"Tue, 26 Feb 2019 13:34:00 GMT","Michael Cohen, President Trump’s former person...",Michael Cohen has been officially disbarred in...,3.0,ABC,3.50,56.75
8,https://abcnews.go.com/Politics/mueller-report...,Katherine Faulders,"Tue, 16 Apr 2019 13:51:00 GMT","As Mueller report release nears, White House p...",While President Donald Trump and the White Hou...,2.0,ABC,-2.33,54.33
9,https://abcnews.go.com/Politics/republican-cen...,Adam Kelsey,"Tue, 26 Feb 2019 20:10:00 GMT",Republican at center of North Carolina electio...,"Mark Harris, the Republican candidate in North...",6.0,ABC,-0.75,53.25


In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/monicabellare/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [19]:
def getPOS(words):
    #return nltk.pos_tag(word)
    return [(word, get_wordnet_pos(word)) for word in words]

In [20]:
def getIndividualPOS(word):
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV} 
    tag = nltk.pos_tag(word[0].upper())
    return tag_dict.get(tag)

In [21]:
#https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    return tag_dict.get(tag, wordnet.NOUN)

In [27]:
print (getPOS(['hello', 'bye']))

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - '/Users/monicabellare/nltk_data'
    - '/anaconda3/nltk_data'
    - '/anaconda3/share/nltk_data'
    - '/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
for index, row in data.iterrows():
    
    #removing stopwords and punctuation
    tokenized = nltk.word_tokenize(row['Body'])
    temp = [word for word in tokenized if word.lower() not in wordsToCut and word not in string.punctuation and word.isalpha()]
    
    #row['Body'] = ' '.join(temp)
    
    #row['Body'] = [word for word in nltk.word_tokenize(row['Body']) if word not in wordsToCut]
    #row['Body'] = row['Body'].lower()
    #nopunc = [char for char in row['Body'] if char not in string.punctuation]
    #nopunc.join('')
    
    #lemmatizing, ONLY KEEPING ENTITIES
    lemmatizer = WordNetLemmatizer()
    partsOfSpeech = getPOS(temp)
    
    lemmatized = [lemmatizer.lemmatize(word, pos) for word, pos in partsOfSpeech if pos == 'n']
    final = ' '.join(lemmatized)
    data.at[index, 'Body'] = final
    
    


In [ ]:
data

In [24]:
vectorizer = TfidfVectorizer(use_idf = True, stop_words = 'english')
matrix = vectorizer.fit_transform(data['Body'])
df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())
#list comprehension
#x = np.where(df['électricité'] != 0)
df

,00,000,00028312038003499,0002831216653206,000lb,0013189,001488,002,003,004,...,électricité,élysée,élysées,émigré,état,île,øystein,über,𝙹𝚞𝚊𝚗,𝚁𝚘𝚍𝚛𝚒𝚐𝚞𝚎𝚣
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.051827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.072702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
'''df = df.drop([columnName for (columnName, columnData) in df.iteritems() if columnName.isalpha() == False], axis=1)
matrix = vectorizer.fit_transform(df)
temp = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())
temp'''
matrix = vectorizer.fit_transform(data['Body'])
temp = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())
temp

,00,000,00028312038003499,0002831216653206,000lb,0013189,001488,002,003,004,...,électricité,élysée,élysées,émigré,état,île,øystein,über,𝙹𝚞𝚊𝚗,𝚁𝚘𝚍𝚛𝚒𝚐𝚞𝚎𝚣
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.051827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.072702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
number_of_clusters = 50
model = KMeans(n_clusters = number_of_clusters)
model.fit(matrix)

# elbow method, silhouette score
# entity recognition: do the same clustering but with only entities (nouns)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=50, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [ ]:
km_scores= []
km_silhouette = []
vmeasure_score =[]
db_score = []
for i in range(2,50):
    km = KMeans(n_clusters=i, random_state=0).fit(matrix)
    preds = km.predict(matrix)
    
    print("Score for number of cluster(s) {}: {}".format(i,km.score(matrix)))
    km_scores.append(-km.score(matrix))
    
    silhouette = silhouette_score(matrix,preds)
    km_silhouette.append(silhouette)
    print("Silhouette score for number of cluster(s) {}: {}".format(i,silhouette))
    

Score for number of cluster(s) 2: -1602.0258908421692
Silhouette score for number of cluster(s) 2: 0.003501963327642399
Score for number of cluster(s) 3: -1591.8255327186903
Silhouette score for number of cluster(s) 3: 0.005383676530458784
Score for number of cluster(s) 4: -1584.1521533716605
Silhouette score for number of cluster(s) 4: 0.006656218318745806


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(km_scores)

In [ ]:
plt.figure(figsize = (15, 10))
plt.scatter(x = [i for i in range(2, 50)], y = km_scores)
plt.xlabel('number of clusters', fontsize=20)
plt.ylabel('elbow method score', fontsize=20)

In [ ]:
plt.figure(figsize = (15, 10))
plt.scatter(x = [i for i in range(2, 50)], y = km_silhouette)
plt.xlabel('number of clusters', fontsize = 20)
plt.ylabel('silhouette score', fontsize = 20)

In [ ]:
number_of_clusters = 50
model = KMeans(n_clusters = number_of_clusters)
model.fit(matrix)

In [ ]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(number_of_clusters):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

In [ ]:
import pandas as pd
gapminder_url='https://bit.ly/2cLzoxH'
gapminder = pd.read_csv(gapminder_url)
gapminder

In [ ]:
gapminder_ocean = gapminder[(gapminder.year > 2000) & (gapminder.continent == 'Oceania')]
gapminder_ocean

In [ ]:
new = gapminder_ocean.drop([columnName for (columnName, columnData) in gapminder_ocean.iteritems() if columnName == 'pop'], axis = 1)
new

In [ ]:
new['69'] = [0, 0, 0, 0]
new

In [ ]:
new = new.drop([columnName if ])

### OLD DATA

In [ ]:
data_old = pd.read_csv('/Users/daviddeng8/Documents/Projects/affinity/data/outlet_data.csv')

vectorizer_old = TfidfVectorizer(use_idf = True, stop_words = 'english')
matrix_old = vectorizer.fit_transform(data_old['content'])
df_old = pd.DataFrame(matrix_old.toarray(), columns = vectorizer.get_feature_names())
df_old

In [ ]:
number_of_clusters_old = 50
model_old = KMeans(n_clusters = number_of_clusters_old)
model_old.fit(matrix_old)

In [ ]:
order_centroids_old = model_old.cluster_centers_.argsort()[:, ::-1]
terms_old = vectorizer_old.get_feature_names()
for i in range(number_of_clusters_old):
    print("Cluster %d:" % i),
    for ind in order_centroids_old[i, :10]:
        print(' %s' % terms_old[ind]),
    print

In [ ]:
km_scores_old= []
km_silhouette_old = []
vmeasure_score_old =[]
db_score_old = []
for i in range(2,50):
    km_old = KMeans(n_clusters=i, random_state=0).fit(matrix_old)
    preds_old = km_old.predict(matrix_old)
    
    print("Score for number of cluster(s) {}: {}".format(i,km_old.score(matrix)))
    km_scores_old.append(-km_old.score(matrix_old))
    
    silhouette_old = silhouette_score_old(matrix_old,preds_old)
    km_silhouette_old.append(silhouette_old)
    print("Silhouette score for number of cluster(s) {}: {}".format(i,silhouette_old))
    

### Spacy Part of speech tagging